In [1]:
import pandas as pd
import numpy as np
import re
import os
import copy
import clgen.clutil
import clgen.model
import sqlite3
import clgen.sampler
import clgen.dbutil
import clgen.explore
import re
import shutil
import os
from subprocess import Popen, PIPE
import glob
import matplotlib.pyplot as plt


In [3]:
def GenerateKernelBatch(noKerns):
    if (os.path.isdir("generatedkernels/")):
        for f in glob.glob("generatedkernels/*.cl"):
            newfname = f.split("/")[-1]
            shutil.move(f, str("CLKernelArchive/"+newfname))
            
    shutil.rmtree("generatedkernels/", ignore_errors=True)
    shutil.rmtree("generatedkernelfeatures/", ignore_errors=True)
    
    os.mkdir("generatedkernels")
    os.mkdir("generatedkernelfeatures")
    
    version = clgen.version()
    print("Successfully loaded CLgen version {}".format(version))

    ###################################
    # Uses Chris Cumins model to generate kernels based on on batch size and max kernels
    # Writes these kernels to a database file
    ###################################
    clgen.clutil.platform_info()
    model = clgen.model.from_tar("./paper-synthesizing-benchmarks-model/model.tar.bz2")
    argspec = ['__global float*', '__global float*', '__global float*', 'const int']
    sampler = clgen.sampler.from_json({
            "kernels": { 
                "args": argspec,
                "max_length": 1000
            },
            "sampler": {
                "batch_size": noKerns * 5,
                "max_kernels": noKerns
            }
        })
    print("Seed text:", clgen.sampler.serialize_argspec(argspec), "\n")
    sampler.cache(model).empty()
    sampler.sample(model)
    db = sampler.cache(model)["kernels.db"]
    num_good_kernels = clgen.dbutil.num_good_kernels(db)
    clgen.explore.explore(db)

    ###################################
    # Establish connection to database file and write the .cl files which pass the rejection filter
    # to a folder called generatedkernels
    ###################################
    conn = sqlite3.connect(db)
    print(conn)
    c = conn.cursor()
    c.execute("SELECT * FROM PreprocessedFiles WHERE status=0;")
    rows = c.fetchall()
    if not(os.path.isdir("generatedkernels")):
        os.mkdir("./generatedkernels")

    for row in rows:
        name = "./generatedkernels/" + str(row[0]) + ".cl"
        newf = open(name, "w+")
        newf.writelines(row[2])
        newf.close()

In [49]:
os.chdir("generatedkernelfeatures/")

In [44]:
totaldat = pd.merge(kernelRuntimesP100, aiwcfeatures, how='inner', on=['size','kernel'])

In [45]:
os.chdir("/workspace/codes/CLFeatureArchive/")
savename = len(os.listdir()) + 1
savefilename = "g" + str(savename) + ".csv"
totaldat.to_csv(savefilename, index=False)

In [4]:
os.chdir("CLFeatureArchive")

In [ ]:
while (True):
    try:
        while (True):
            os.chdir("/workspace/codes")
            GenerateKernelBatch(10)
            os.chdir("/workspace/codes/generatedkernelfeatures/")

            aiwcfeatures = pd.DataFrame()
            kernelRuntimesP100 = pd.DataFrame()

            for psize in ['tiny','small','medium', 'large']:
        #     psize = 'tiny'

                for kernal in glob.glob("../generatedkernels/*.cl"):
    #                     try:
                    print("generating features for: ", psize, " ", kernal)
                    process = Popen(["../../../oclgrind/bin/oclgrind", "--workload-characterisation", "../synthetic-benchmark-driver/sbd", kernal, psize, "0", "0", "aiwc"], stdout=PIPE)
                    (output, err) = process.communicate(timeout=60)
                    exit_code = process.wait()
                    print("Finished generating AIWC features")

                    for f in glob.glob('aiwc_A*.csv'):
                        try:
                            currFeatures = pd.read_csv(f).values.T
                            a = pd.DataFrame(data=currFeatures[1:,1:], columns=currFeatures[0,1:])  # 1st row as the column names
                            a['size'] = psize
                            a['kernel'] = kernal
                            a['application'] = "CLgen Program"
                            aiwcfeatures = pd.concat([a, aiwcfeatures])
                        except:
                            continue
                        break

                    for f in glob.glob('aiwc_A*.csv'):
                        os.remove(f)

                    for f in glob.glob('lsb.*'):
                        os.remove(f)

                    print("gathering runtime data for: ", psize, " ", kernal)
                    for i in range(0, 10):
                        process = Popen(["../synthetic-benchmark-driver/sbd", kernal, psize, "0", "0", "runtime"], stdout=PIPE)

                        (output, err) = process.communicate(timeout=60)
                        exit_code = process.wait()

                        try:
                            kruntime = pd.read_csv("lsb.sbd.r0", skiprows=9, sep="[ ]+", engine='python', skipfooter=1)
                            kruntime['size'] = psize
                            rt = copy.deepcopy(kruntime[kruntime['region'] == 'kernel'])

                            kernelRuntimesP100 = pd.concat([kernelRuntimesP100, rt])
                        except:
                            print("fail wail")

                        for f in glob.glob('lsb.*'):
                            os.remove(f)
                        for i in glob.glob('aiwc_A*.csv'):
                            os.remove(f)
    #                     except:
    #                         print("Something randomly failed")
    #                         os.remove(f)

                    for f in glob.glob("aiwc*"):
                        os.remove(f)

            totaldat = pd.merge(kernelRuntimesP100, aiwcfeatures, how='inner', on=['size','kernel'])

            os.chdir("/workspace/codes/CLFeatureArchiveAll/")
            savename = len(os.listdir()) + 1
            savefilename = "g" + str(savename) + ".csv"
            totaldat.to_csv(savefilename, index=False)
    except:
        print("Woot")

Successfully loaded CLgen version 0.1.7
Platform:   Linux
Memory:     95227 MB

Device:     None

Device:     GPU Tesla P100-PCIE-12GB
Compute #.: 56
Frequency:  1328 HZ
Memory:     12198 MB
Driver:     418.87.00

Device:     CPU Intel(R) Xeon(R) Gold 6134 CPU @ 3.20GHz
Compute #.: 32
Frequency:  3200 HZ
Memory:     95227 MB
Driver:     18.1.0.0920
Seed text: __kernel void A(__global float* a, __global float* b, __global float* c, const int d) { 

beginning batch 1 ...


/* === SAMPLE 1 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < d) {
    c[e] = a[e] * b[e] - d;
  }
  b[e] = 0;
}

/* === SAMPLE 2 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  if (e >= d) {
    return;
  }
  c[e] = (f + e) / (b[e] * c[e]);
}

/* === SAMPLE 3 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {


    const int j = min(h - 1, 0) + e;
    const int k = min(h + 1, i - 1);

    const int l = j - (i & 1);
    const int m = g.y;
    const int n = m % b;
    const int o = l * b + f;
    const int p = l * p + q * e;

    const int q = h + p * d + k;

    const int r = i - p + 1;

    const int s = h * b + c;
    a[s] = a[s] - a[s];
  }
}

/* === SAMPLE 27 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f = get_global_id(1);

  int g = e + d * f;

  if (b[f] != 0) {
    get_local_BOXTH;
    i.val = d;
  }

  if (f == 0) {
    g++;
    g++;
  }
  c[g] = h;
}

/* === SAMPLE 28 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  for (int f = 0; f < 10000; f++) {
    c[d] = a[d] + b[d - 16];
    barrier(1);
    for (int h = b - e; h > 0; h >>= 1) {
      if (d < h) {
        int g = d - (d & 1);
        for (int h = b - 1; h >= 0; h--) 

    f = d * (e - g * c * e + f);

    e -= b[d];
    if (!e && f[c <= h]) {
      b[f] = 0;
    }
  }
}

/* === SAMPLE 46 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < d)
    e[d + e] = a[e];
  else
    b[d] = a[e] + 0xffffffff;
  b[e] = d[e] * a[e];
}

/* === SAMPLE 47 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < d) {
    m[e] += b[e];
  }
}

/* === SAMPLE 48 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f = get_global_id(1);
  int g = get_global_size(1);
  int h = get_global_size(1);

  int ai = A(f, b);
  int aj = e + f - 1;

  if (k < f && al < (f - h)) {
    int j = i + g;
    ah = aj - e;
    al = (b * l) / i;
    float k = (f + j + 1) * b;
    ad = al + (((j + j) << 5) - 1);
  }

  if (b < 0) {
    g[h] = c[j];
    d[h] 

generating features for:  small   ../generatedkernels/18fc5e95d0ec19ccf58bcbee0decf9f13e2e8e9e.cl
Finished generating AIWC features
gathering runtime data for:  small   ../generatedkernels/18fc5e95d0ec19ccf58bcbee0decf9f13e2e8e9e.cl
generating features for:  medium   ../generatedkernels/f2ba75bf8bdcfd54964f676e36e6a1a438266656.cl
Finished generating AIWC features
gathering runtime data for:  medium   ../generatedkernels/f2ba75bf8bdcfd54964f676e36e6a1a438266656.cl
generating features for:  medium   ../generatedkernels/27edf81dc020de1724621998197059c1951d6adc.cl
Finished generating AIWC features
gathering runtime data for:  medium   ../generatedkernels/27edf81dc020de1724621998197059c1951d6adc.cl
generating features for:  medium   ../generatedkernels/af2b23638940e4a54782d34757edf75cd5223e2d.cl
Finished generating AIWC features
gathering runtime data for:  medium   ../generatedkernels/af2b23638940e4a54782d34757edf75cd5223e2d.cl
generating features for:  medium   ../generatedkernels/4442829

    for (size_t m = 0; m < k; m++) {
      float4 n = read_imagef(a, g, (int2)(gm .x - a, h, k)).x - read_imagei(b, a, d}).x / get_local_size(1);
      float4 o = read_imagef(a, h, (int2)(gn, hsc));

      n.x = b[h];
      o.x = n[0].x - (f[h].x - 

/* === SAMPLE 8 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  if (e < d) {
    float f = b[e];
    float g = d[f];

    a[e] = a[f];
  }

  return;
}

/* === SAMPLE 9 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f = get_global_id(1);
  int g = e * c + d;

  if (g < c && f < c) {
    e += d;
  }
  b[e] = a[0] + a[e];
}

/* === SAMPLE 10 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e >= d) {
    return;
  }
  float4 f = (float4)(1.0f, 1.0f, 1.0f, 1.0f);
  float4 g = (float4)(1.0f, 1.0f, 1.0f, 3.

    aj = 0;

  if (aq > 0) {
    ar = -1;
    as = -1;
    at = 0;
    aw = 1;
  }
  if (ar <= 10) {
    as = ao;
    az = 1;
  }
  barrier(1);

  for (u = 0; u < aa; ++u) {
    int ah = u - 1;
    int ai = ad & (g - 1);

    float aj = ae - af;
    float ak = ac - ak;

    f

/* === SAMPLE 28 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < c) {
    a[e + c] = b[e];
  }
  c[e] = e + 1;
  barrier(1);

  a[b + f] = c[e * 16 + d];
}

/* === SAMPLE 29 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f = get_global_id(1);
  int g = 0;
  int h = 0;
  if (h == 1 && c[h] == 1) {
    ghatt g = -1;
    int h = 0;
    int i = 0;

    for (g = -e; g <= g; ++g) {
      for (f = 0; f < e; f++) {
        const int h = (g - 1 - f) * e;

        for (int m = 0; m < 7; m++) {
          float4 n = read_imagef(a, e, (int2)(h + b, h + klx i 

  int f = get_global_id(1);
  int g = g / d;

  if (b[g] * d + e)
    a[f] = f;
}

Number of content files:            48

Number of unique content files:     48 (100%)
Total content line count:           824
Content file line counts:           min: 6, med: 11, avg: 17, max: 56

Number of good preprocessed files:  6 (12%)
Lines of good preprocessed code:    43 (5%)
Good preprocessed line counts:      min: 6, med: 7, avg: 7, max: 8

beginning batch 2 ...


/* === SAMPLE 1 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  if (e < d) {
    c[e] = a[e] + b[e];
  }
  b[e] = (char)a;
}

/* === SAMPLE 2 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  int f = e - d;
  int g = e > b ? a : 1;
  int h = d < 0 ? h : d;
  int i = f - h;
  int j = h / 2;
  int k = i + h * 1;

  int l = 0, m = 0;
  int n;
  if ((e >> 3) >= 12 - 1)
    return;

  in

  float ah = 

/* === SAMPLE 21 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < d) {
    int f = c * c + e;
    for (int g = 0; g < c; g++) {
      for (f = 0; f < f; f++) a[f] += e[f];
    }
    if (a[f] <= a[e])
      a[f] = 1;
  }
  barrier(1);

  b[2 * e] = b[2 * e];
  b[3 * f] = b[1 + f * f + 2];
  b[2 * f + 1] = b[2 * f + 1];

  barrier(1);

  for (int h = 0; h < 3; h++) {
    c[h] = a[h];
    h += a[h];
    h += a[i];
  }

  return;
}

/* === SAMPLE 22 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  c[e] = a[e] + b;
}

/* === SAMPLE 23 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < c) {
    b[e] = 0.0f;
  }
}

/* === SAMPLE 24 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e 

__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < d) {
    a[e + c] = b[e];
  }
}

Number of content files:            95

Number of unique content files:     95 (100%)
Total content line count:           1,521
Content file line counts:           min: 5, med: 9, avg: 16, max: 56

Number of good preprocessed files:  20 (21%)
Lines of good preprocessed code:    141 (9%)
Good preprocessed line counts:      min: 5, med: 7, avg: 7, max: 10

beginning batch 3 ...

Number of content files:            95

Number of unique content files:     95 (100%)
Total content line count:           1,521
Content file line counts:           min: 5, med: 9, avg: 16, max: 56

Number of good preprocessed files:  20 (21%)
Lines of good preprocessed code:    141 (9%)
Good preprocessed line counts:      min: 5, med: 7, avg: 7, max: 10

generating features for:  tiny   ../generatedkernels/db7951d4a742cb8ffce284fde06d9b4bdcdb77fe.cl
Finish

generating features for:  small   ../generatedkernels/54ad47b6316017a85b431efa62516712d5d229f7.cl
Finished generating AIWC features
gathering runtime data for:  small   ../generatedkernels/54ad47b6316017a85b431efa62516712d5d229f7.cl
generating features for:  small   ../generatedkernels/bf54516a7774d46d8deaffc21b04893c41c10ae5.cl
Finished generating AIWC features
gathering runtime data for:  small   ../generatedkernels/bf54516a7774d46d8deaffc21b04893c41c10ae5.cl
generating features for:  small   ../generatedkernels/7a44c16983dccf01c0355049fe7f7316b3bc2afa.cl
Finished generating AIWC features
gathering runtime data for:  small   ../generatedkernels/7a44c16983dccf01c0355049fe7f7316b3bc2afa.cl
generating features for:  small   ../generatedkernels/a9e75d1265b17e50e5cabb641d692bd565f7d418.cl
Finished generating AIWC features
gathering runtime data for:  small   ../generatedkernels/a9e75d1265b17e50e5cabb641d692bd565f7d418.cl
generating features for:  small   ../generatedkernels/f4853512121ac5

In [ ]:
GenerateKernelBatch(1000)

Successfully loaded CLgen version 0.1.7
Platform:   Linux
Memory:     95227 MB

Device:     None

Device:     GPU Tesla P100-PCIE-12GB
Compute #.: 56
Frequency:  1328 HZ
Memory:     12198 MB
Driver:     418.87.00

Device:     CPU Intel(R) Xeon(R) Gold 6134 CPU @ 3.20GHz
Compute #.: 32
Frequency:  3200 HZ
Memory:     95227 MB
Driver:     18.1.0.0920
Seed text: __kernel void A(__global float* a, __global float* b, __global float* c, const int d) { 

beginning batch 1 ...


/* === SAMPLE 1 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int m = get_global_id(1);

  if (m < g && l < g && l < h) {
    int o = l * j + l;
    int p = n * b;

    for (int q = 0; q < i; q++) {
      float r = 0.0f;
      for (int s = p; s <= n; s++) {
        float t = e[t];
        float u = d[s + p * b + s];

        if (d < p)
          c[t * j + s] = (r + s) / l[t + j];
      }

      barrier(1);
      p /= sizeof(float);

      f

__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < c) {
    b[e] = a[e] * d[e] * a[e];
  }
}

/* === SAMPLE 28 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0) * f.x;
  int f = c.y * c.x + d.y * c.x + d.y * e.y + d.y * e.y + e.y * c.y + e.y;

  for (f = 0; f < 10; f++) {
    e += a[f];
  }
  a[f] = h;
}

/* === SAMPLE 29 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < c) {
    a[e % 32] = a[e * 4 + 2];
    a[e * 2 + 2] = c[2 * c];
  }
  b[e] = a[e * 3];
}

/* === SAMPLE 30 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < d) {
    c[e] = a[e] * b[e] + 0xF;
  }
}

/* === SAMPLE 31 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int 

  int e = get_global_id(0);
  if (e >= d) || e >= d) {
    return;
  }

  int f = e - d;
  int g;
  f = (b - d) * 0.0625f / b;

  return a;
}

/* === SAMPLE 57 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e >= i)
    return;

  b[e] = c[0];
}

/* === SAMPLE 58 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < d) {
    const int f = e + c + 1;
    const int g = h.x;
    const int h = get_global_id(0) / c;
    const int i = f - b;
    const int j = f + e;
    const int k = j;
    const int l = h - f * b;

    const int m = l * f + i;

    if (l < g && m >= d) {
      j = a[k];
      g = 0.0;
      h = k[j];
      if (l < 0)
        l += j;
      else if (l * k <= w) {
        j = fmin(g[l], f[n]);
        j = min(l, l);
      }
      if (k == 0) {
        int r = m * b;
        for (int s = p; s < q; s++) {
          fl

      for (int l = 0; l < 4; l++) {
        float m = 0;
        float n = a[j + i];
        float o = d[l];
        float p = g[l];
        if (l > j)
          e[l] = 0.0;
        for (int p = 0; p < 0; p++) {
          for (int q = 0; q < 128; q++) n += d[q] * g[q];
        }
        m[i >> 1] = n;
      }
      barrier(1);
    }

    if (j > e - 1)
      q = f[p];
    else if (i == 0)
      b[j] = t - (r[i] + j[i] + q * n[i] - n[p]);
  }

  if (j > 0 && j < d)
    b[h] = 0;

  for (int r = 0; r < 100; r++) {
    if ((c[s] == q[r]) && (q[r] == o && r == 2))
      s = q + ((16 - a) * h);
    else
      s = ((1 << 1) - 1) & (l);
    else if (u == 0)
      c[n] = 0.0f;
    else
      b[i] = 0.0f;
    else
    

/* === SAMPLE 83 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  float f = 0.f;

  float g = 3.14159265358979323846f * f.z;

  b[c] = h;
}

/* === SAMPLE 84 === */	
__kernel void A(__global float* a, 

  size_t m = get_global_id(1) + 2;

  const int a = b + a * g;
  const int n = m + (n - 1) / 2;

  if (n < c && l > 7 - 2)
    return;

  int q = 0;
  int r = 0;
  int s = h[s];
  float t = 1.0f - (k + 0.5f) * i;
  float u = d[j] / sqrt(1.0f - t) / 2.0f;
  int v = (int)get_local_id(0);
  for (int w = 0; w < h; ++w) {
    for (int aa = 0; aa < ab; ++af) {
      for (int o = -4; o <= 4; ++o) {
        int ac = aa / r;
        int ad = q + x * h * p;
        float ae = x + ac;

        float af = (aa < 0) ? 0 : af;
        int ag = ab << 1;
        float4 ah = (af >= 0 && ah >= 0 && ah < g) ? -ab : (af + v);

        float4 ai = o[ac + r];

        ab = (ae - (r + ad) * af) / af;
        af = ah * ai;
        ah = ai * ai;
        ah = -z - ai * ah;
        ah = ai * ai;
        ai = ah * l - m * ad;
        ah = ah * l - ai * ai;
        ai = ah * l + y;

        aj = ah *

/* === SAMPLE 102 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
 

...

  ap.z = ap.y - as;
  ap.z = ar ? ap : (ar - 4);

  ap[mad24(3u, ao, ao.x)] = ar;
  an[aq.y*3] = su - an;

  an = E(aj.x, am.x, an.x, an.x * ao, ap, an.x, 0, &aq, &ao);

  int aq = (al < as);
  aq =

/* === SAMPLE 568 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < c)
    e[e] = a[e] + b;
}

/* === SAMPLE 569 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f = get_global_id(1);

  if (f < ab && f < av - 1 && g < av - 1 && g < av || e >= ay)
    return;

  int g = ah[c];
  int ao = af[as];
  int ap = 1 << ((ao / ae) + 2);
  int av = (aw % aa) * as;
  int aw = 0;
  int x, y = 0, az = 0;
  int z = -1;
  while (x < v | z > 0) {
    s = ab[at] * al;
    b[v] = n;
  }
}

/* === SAMPLE 570 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < d) {

    a[e] = 0;
  }
}

/* === SAMPLE 590 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  if (e < d) {
    uint f = c * d;
    uint g = h + 0;
    uint h = f * g;

    uint m = c + f;
    a[f] = -i;
  }
}

/* === SAMPLE 591 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f = get_global_id(1);
  int g = e + d * f;

  for (int h = 0; h < c; ++h) {
    const int i = (h + f) < f ? 0 : f;
    const int j = e;
    const int k = i * e;
    const int l = i + j / c;

    float o = 0.0;
    int p = 1;
    float q = 0.0;
    for (int r = q; r < r; ++r) {
      __global float s[16 * 64];
      int t[32 * 64];

    b[f / 4 + g] = l[c[q * 32 + r]];
    t += j[o + p];

    t[u * 17 + s] = h[t * 17 + s];
  }
}

void D(__global float* a, __constant float* b, __global float* p, int r, int p, int q, int r, int s, int u) {
  float e = sqrt(f);
  float

  sampler_t f = 0x0000 | 0x0002 | 0x0000;
  float4 g = read_imagef(a, f, (int2)(c, d));

  float4 h = read_imagef(a, e, (int2)(f2, g));

  float4 k = read_imagef(c, d, g);
  float4 l = read_imagef(b, d, (int2)(f1, g));
  float4 m = read_imagef(b, j, (int2)(0, -1));
  float n = read_imagef(a, d, g).s0;
  float4 o = read_imagef(b, a, (int2)(2 * f + 1, 2 * h + 1));
  float4 p = read_imagef(b, a, (int2)(2 * f + 1, 2 * i));

  int2 q = (int2)(d + h, i);
  int2 r = (int2)(get_global_id(0), get_global_id(1));
 write_imagef(c, (int2)(g, h), r);
}

/* === SAMPLE 617 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  if (e >= d) {
    return;
  }
  return e;
}

/* === SAMPLE 618 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int m = get_global_id(1);

  int f = get_global_size(0);
  int n = get_global_size(1);
  int o = get_global_size(2);

  f

    int g = i * d + i;
    int h = g * d + f;
    int i = c + f;

    if (h < 0) {
      for (int j = -e; j <= d; j++)
        for (int k = 0; k < 8; ++k) {
        e[k * g] = 0;
      }

      a[e] += b[k * 16 + j];
    }
  }
}

/* === SAMPLE 644 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < c)
    e[e] = a[e] + b[e];
}

/* === SAMPLE 645 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < c)
    e[e] = a[e] * a[e];
}

/* === SAMPLE 646 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f = get_global_id(1);

  if (f < c) {
    float g = n[e];

    float h = e[f];
    float4 i = b[f];

    for (int j = 0; j < e; j++) {
      float4 k = a[j];
      float4 l = a[e];

      for (int m = 0; m < k; m++) {
        float4 n = read_imagef(a, g, (

  float h;
  for (e = 0; e < d; e++) {
    h = h * h;
    float j = c;

    if (c > j)
      f = f;

    float m = e * d[n] - f[j];
    float n = f[m].x * a[k].x - g.x * a[l].x - h.x * a[k].x * c[k].x - a[k].x * a[k].x * a[k].x - g[k].x * a[k].x * a[k];

    d += f;
    k = d * m + a[m].y * a[k];
    j *= e.z;
    j.z += o.y * o;
    o.y += p.x * o;
    p.x += i.y * l;
    o.z += l.x * e;
    o.z += o.x * l;
    p.x += p.x * p.x + p.y * q.x + p.y * r.x + p.z * r.x + p.w * s.x;
    k.y += (r / 0.5);
    p.y += p.y;
    p.y += o.y;
    p.z += o.z;
    p.z += o.y;
    p.y += o.x;
    p.z += o.y;
    p.z += o.x;
    p.z += (l + 0.5) * m;
  }

  if (p < 100 || p >= g - 1) {
    o = a[0] * 2.0f - j;
    q = r - s;
    d[j] /= -0.00001f;
    return;
  }

  float4 y;
  float4 aa;
  float4 ab;

  {
    y = abs(a

/* === SAMPLE 669 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e >= d) {
    return;
  }
  int f =

In [12]:
for f in glob.glob("generatedkernels/*.cl"):
    currfile = []
    for l in open(f, "r+"):
        currfile.append(l)
    if (len(currfile) > 12):
        for line in currfile:
            print(line)